In [ ]:
import sys
print(sys.executable)

c:\Users\Rubah\Downloads\Final_Project\insight_env\Scripts\python.exe


: 

In [48]:
import random
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. Setup Folder and Save Path
# Ensuring it saves in the "../Data/" directory relative to your script
save_dir = os.path.join("..", "Data")
os.makedirs(save_dir, exist_ok=True)
CSV_FILE = os.path.join(save_dir, "etimad_tenders_full.csv")

# 2. Scraper Configuration
MAX_PAGES = 1500  # Target for Project Nabaha
options = webdriver.ChromeOptions()
options.add_argument('--window-size=1920,1080')
# options.add_argument('--headless') # Uncomment if you want to run in background

# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_tenders = []
seen_refs = set() # Set to track unique Reference Numbers and prevent duplicates

try:
    for page_num in range(1, MAX_PAGES + 1):
        # Target the page number directly via URL parameter
        url = f"https://tenders.etimad.sa/Tender/AllTendersForVisitor?pageNumber={page_num}"
        print(f"🚀 Navigating to Page: {page_num}...")
        driver.get(url)

        try:
            # Wait until tender cards are present (Confirming page load)
            WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "card"))
            )
            
            # Random delay to mimic human behavior and avoid bot detection
            time.sleep(random.uniform(4, 7))

            cards = driver.find_elements(By.CLASS_NAME, "card")
            if not cards:
                print(f"🛑 No more data found at page {page_num}. Ending loop.")
                break

            for card in cards:
                try:
                    # Extract the Detail URL
                    link = card.find_element(By.XPATH, ".//a[contains(@href, 'Tender')]").get_attribute("href")
                    
                    # Extract the Reference Number from the card text
                    full_text = card.text
                    ref_number = "N/A"
                    
                    # Parsing lines to find the Arabic label for Reference Number
                    for line in full_text.split('\n'):
                        if "الرقم المرجعي" in line:
                            # Split by colon or replace text to isolate the number
                            ref_number = line.split(":")[-1].strip() if ":" in line else line.replace("الرقم المرجعي", "").strip()
                            break
                    
                    # Deduplication check: Only add if Reference Number is new
                    if ref_number not in seen_refs:
                        seen_refs.add(ref_number)
                        all_tenders.append({
                            "Page": page_num,
                            "Reference Number": ref_number,
                            "URL": link,
                            "Scraped At": pd.Timestamp.now()
                        })
                except Exception:
                    continue # Skip individual card if it fails

            # Periodic Save: Every 5 pages to ensure data isn't lost if crash occurs
            if page_num % 5 == 0:
                pd.DataFrame(all_tenders).to_csv(CSV_FILE, index=False, encoding='utf-8-sig')
                print(f"💾 Milestone: Saved {len(all_tenders)} total unique tenders so far.")

        except Exception as e:
            print(f"⚠️ Page {page_num} timed out or failed. Skipping to next page...")
            continue

finally:
    # Final save and cleanup
    if all_tenders:
        pd.DataFrame(all_tenders).to_csv(CSV_FILE, index=False, encoding='utf-8-sig')
        print(f"🏁 Mission Complete! Final count: {len(all_tenders)} unique tenders.")
    
    driver.quit()
    print(f"📁 File saved at: {CSV_FILE}")

🚀 Navigating to Page: 1...
🚀 Navigating to Page: 2...
🚀 Navigating to Page: 3...
🚀 Navigating to Page: 4...
🚀 Navigating to Page: 5...
💾 Milestone: Saved 30 total unique tenders so far.
🚀 Navigating to Page: 6...
🚀 Navigating to Page: 7...
🚀 Navigating to Page: 8...
🚀 Navigating to Page: 9...
🚀 Navigating to Page: 10...
💾 Milestone: Saved 60 total unique tenders so far.
🚀 Navigating to Page: 11...
🚀 Navigating to Page: 12...
🚀 Navigating to Page: 13...
🚀 Navigating to Page: 14...
🚀 Navigating to Page: 15...
💾 Milestone: Saved 90 total unique tenders so far.
🚀 Navigating to Page: 16...
🚀 Navigating to Page: 17...
🚀 Navigating to Page: 18...
🚀 Navigating to Page: 19...
🚀 Navigating to Page: 20...
💾 Milestone: Saved 120 total unique tenders so far.
🚀 Navigating to Page: 21...
🚀 Navigating to Page: 22...
🚀 Navigating to Page: 23...
🚀 Navigating to Page: 24...
🚀 Navigating to Page: 25...
💾 Milestone: Saved 150 total unique tenders so far.
🚀 Navigating to Page: 26...
🚀 Navigating to Page: 27

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os

# 1. Setup Folder and Paths
save_dir = os.path.join("..", "Data")
os.makedirs(save_dir, exist_ok=True)
INPUT_CSV = os.path.join(save_dir, "etimad_tenders_full.csv")
OUTPUT_CSV = os.path.join(save_dir, "nabahan_full_details.csv")

target_fields = [
    "اسم المنافسة", "رقم المنافسة", "الرقم المرجعي", "الغرض من المنافسة",
    "قيمة وثائق المنافسة", "حالة المنافسة", "مدة العقد", "هل التأمين من متطلبات المنافسة",
    "نوع المنافسة", "الجهة الحكوميه", "الوقت المتبقى", "طريقة تقديم العروض",
    "مطلوب ضمان الإبتدائي", "الضمان النهائي", "رقم الترسية", "آخر موعد لإستلام الإستفسارات",
    "آخر موعد لتقديم العروض", "تاريخ فتح العروض", "تاريخ فحص العروض", "فترة التوقف",
    "التاريخ المتوقع للترسية", "تاريخ بدء الأعمال / الخدمات", "بداية إرسال الأسئلة و الاستفسارات",
    "اقصى مدة للاجابة على الاستفسارات", "مكان فتح العرض", "مجال التصنيف", "مكان التنفيذ",
    "التفاصيل", "نشاط المنافسة", "تشمل المنافسة على بنود توريد", "أعمال الإنشاء",
    "أعمال الصيانة والتشغيل", "آليات المحتوى المحلي المطبقة في المنافسة"
]

def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    
    # --- FIX START: BYPASS INTERNET ---
    try:
        # Try to use the manager, but if it fails (offline), use local default
        service = Service(ChromeDriverManager().install())
    except Exception:
        print("🌐 Offline Mode: Using local Chrome driver...")
        service = Service() # This looks for 'chromedriver' in your PATH or environment
    # --- FIX END ---
    
    return webdriver.Chrome(service=service, options=options)

tabs_names = ["المعلومات الأساسية", "العناوين والمواعيد المتعلقة بالمنافسة", "مجال التصنيف وموقع التنفيذ والتقديم", "إعلان نتائج الترسية"]

# 2. Load URLs
try:
    df_input = pd.read_csv(INPUT_CSV)
    urls = df_input["URL"].tolist()
    print(f"✅ Found {len(urls)} URLs. Starting...")
except Exception as e:
    print(f"❌ Error loading file: {e}")
    exit()

all_tenders_data = []
driver = create_driver()

# 3. Processing Loop (For all URLs)
for i, url in enumerate(urls):
    try:
        print(f"🔍 [{i+1}/{len(urls)}] Processing: {url}")
        driver.get(url)
        
        # Wait for page content
        WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        tender_info = {"الرابط": url}

        for tab in tabs_names:
            try:
                xpath = f"//a[contains(normalize-space(),'{tab}')] | //span[contains(normalize-space(),'{tab}')]"
                tab_btn = WebDriverWait(driver, 7).until(EC.element_to_be_clickable((By.XPATH, xpath)))
                
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab_btn)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", tab_btn)
                time.sleep(3) 

                soup = BeautifulSoup(driver.page_source, "html.parser")
                
                # Data Extraction Logic
                for item in soup.find_all("li", class_="list-group-item"):
                    title = item.find("div", class_="etd-item-title")
                    info = item.find("div", class_="etd-item-info")
                    if title and info:
                        key = title.get_text(strip=True).replace(":", "").strip()
                        val = info.get_text(strip=True).replace("...عرض الأقل...", "").strip()
                        if val:
                            tender_info[key] = val

                for table in soup.find_all("table"):
                    table_title = table.find_previous(["h4", "h5", "div"], class_="text-primary")
                    title_text = table_title.get_text(strip=True) if table_title else f"جدول_{tab}"
                    rows = table.find("tbody").find_all("tr") if table.find("tbody") else []
                    content = ["[" + " - ".join(td.get_text(strip=True) for td in r.find_all("td")) + "]" for r in rows]
                    if content:
                        tender_info[title_text] = " | ".join(content)

            except Exception:
                continue 

        all_tenders_data.append(tender_info)

    except Exception as e:
        print(f"⚠️ Skipping URL {i+1} due to error.")
        continue

    # 4. Periodically save and refresh driver to avoid memory leaks
    if (i + 1) % 20 == 0:
        pd.DataFrame(all_tenders_data).to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
        print(f"💾 Progress saved at {i+1} rows.")
        driver.quit()
        driver = create_driver()

# 5. Final Save
df_results = pd.DataFrame(all_tenders_data)
cols = ["الرابط"] + [f for f in target_fields if f in df_results.columns]
remaining = [c for c in df_results.columns if c not in cols]
df_results = df_results.reindex(columns=cols + remaining)

df_results.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
driver.quit()
print(f"🏁 DONE! Saved to: {OUTPUT_CSV}")

✅ Found 8850 URLs. Starting...
🔍 [1/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=GxOh24u1stmOoDzIpcI2EA==
🔍 [2/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=KBMkaY*@@**1ceG6kmglUt3FRA==
🔍 [3/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=bIu%20vOCLF%20Xbqxt8x*@@**9Xgw==
🔍 [4/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Gm*@@**GSWgbPJ*@@**BBbWc8w4QNg==
🔍 [5/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=BwxmAaxud*@@**xqy*@@**a0d%20lWyg==
🔍 [6/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=d%20zLhfsATY78xT2rotrHAA==
🔍 [7/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Rnj97er%20LYISMNR3X3dfvQ==
🔍 [8/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=kBt3kEv*@@**8OW3J5ZWnj82kQ==
🔍 [9/8850] Processing: https://tenders.etimad.sa/Tender/Det

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


🔍 [21/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=anTVLcjRFWEFrDGdLoYZAg==
🔍 [22/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=MxyJPRj0MYRrasMq4Jt3Aw==
🔍 [23/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=rwcRAep7BW9CsFimowYzUg==
🔍 [24/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=YX2oLaduKe0l6nZk7NQlQg==
🔍 [25/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=ueXwCbfDxpH%20rzr7MuNhlQ==
🔍 [26/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=viZsT40QIe4ndr0zev*@@**8UQ==
🔍 [27/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=vv8vg*@@**9JZisMEZQuxFgiOQ==
🔍 [28/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=VzjE4jYKihn34N2Nh3f3pA==
🔍 [29/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=cRuYDA882WvDWrjFXSt1fQ==


There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


🔍 [261/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=gsZZVioS9itB7M%20%206fpp7A==
🔍 [262/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=PKYRyHPRCgT7kGFWmRe7QQ==
🔍 [263/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=hM4Fqpt4IDfyZaj*@@**h1Dt4g==
🔍 [264/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=OZKHDEZ9XvDvrd1wAMlTiw==
🔍 [265/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=ai3FYHo5I0n7FsOL3mCzng==
🔍 [266/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=jKm96a572clG*@@**QYk6Yacog==
🔍 [267/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=%20qZk%20rhIAql3bD2fvOuQDQ==
🔍 [268/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=oSCdsJUt%20iIOcbTZFB0N1w==
🔍 [269/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=JgC%20vY

...

🔍 [2181/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=owrRxFKqJ53ZxmCAd%20iKpg==
🔍 [2182/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Ldp5myEyf6EKpCSw2bTK9A==
🔍 [2183/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=bB0cxeYDtb9r7eNopG3eaw==
🔍 [2184/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=TyAOKOLUkxyt29F13WLTYw==
🔍 [2185/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=XDO%20A6gNCUNMc9Yuzs0xYQ==
🔍 [2186/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=GUzmn7Xh6mqkBYm1nxp*@@**Rw==
🔍 [2187/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=k9a07vctLEx%20rejo8IRO5Q==
🔍 [2188/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=5CbeEO*@@**v0NDJb78KikJB7g==
🔍 [2189/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=1vm

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


🔍 [2201/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Ky0D8ISkoBh0xulAucE7wA==
🔍 [2202/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=RkC9e21LmlxqH2T9CsyiSA==
🔍 [2203/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=R*@@**6tW3qjFqnYQJyud5NhKA==
🔍 [2204/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Zj2FTmlhxugoRbbyYpFU8g==
🔍 [2205/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Pq1xL4MOPKJplsm*@@**1HSXDA==
🔍 [2206/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=Kn3DP01iquIazfGCSIyu2Q==
🔍 [2207/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=eXzGaYpyJwtLoSJBbPAEOw==
🔍 [2208/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=H*@@**fTZp2vRLSifnVL4lKUAw==
🔍 [2209/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=a4pPH

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


🔍 [2221/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=lYlftYaYTI6PiyE3%20ucZvg==
🔍 [2222/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=cugivwbip8AGdihn4khsyw==
🔍 [2223/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=YJflmwuXLxdI1ks4MyhcZg==
🔍 [2224/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=6EBXi2fOvbYiCpAPXW3sGg==
🔍 [2225/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=JXYLjGj5rzx8xa2kbZoagQ==
🔍 [2226/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=WVVZlgoAYFEB0Zs2JUguSg==
🔍 [2227/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=GjlNvOTQ%20DsHrBfiTR5eag==
🔍 [2228/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=d26eMY4WsYaGxnq04PG%20QQ==
🔍 [2229/8850] Processing: https://tenders.etimad.sa/Tender/DetailsForVisitor?STenderId=vUC4CpwwHvf